In [115]:
import plotly.graph_objs as go
import numpy as np
from ipywidgets import interact, FloatSlider, Layout

In [116]:
def f(x, y):
    return x + y

R = 8.314

pMin, pMax = 0, 60000
tC, tH = 300, 2300

vF = 1000

pF0 = 6000
tF0 = 600
pT0 = 8000
tT0 = 700

def moles(p, v, t):
    return (p*v)/(R*t)

def calculateSurface(pF, tF, pT, tT):
    nF = moles(pF, vF, tF)
    nT = moles(pT, vF, tT)
    
    nR = np.linspace(0, nF, N)
    tI = np.linspace(tC, tH, N)
    nR, tI = np.meshgrid(nR, tI)
    nI = (tT*nT-tF*(nF-nR))/tI
    
    return nR[0,:], tI[:,0], nI

def calculateEmbedding(pF, tF, pT, tT):
    pass

def mask(nF, nR, tI, nI):
    return np.logical_or.reduce([
        nR < 0,
        nR > nF,
        tI < tC,
        tI > tH,
        nI < 0
    ])

In [166]:
fig = go.FigureWidget()
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=-1.7, y=1.7, z=0.5))
fig.update_layout(
    height=800,
    scene_camera=camera,
    scene_aspectmode='cube',
    scene = dict(
        xaxis = dict(title='Moles removed (nR)'),
        yaxis = dict(title='Temperature added (tI)'),
        zaxis = dict(title='Moles added (nI)')))

fig.add_surface(opacity=0.5, showscale=False)
fig.add_scatter3d(mode='lines', line=dict(color='black'), name='Embedding')
fig.add_scatter3d(mode='markers', marker=dict(color='green'), name='nR=0')
fig.add_scatter3d(mode='markers', marker=dict(color='red'), name='tI=tH')
fig.add_scatter3d(mode='markers', marker=dict(color='blue'), name='tI=tC')

S = fig.data[0]
E = fig.data[1]
A = fig.data[2]
B = fig.data[3]
C = fig.data[4]

SN = 30
EN = 200

def point(handle, nR, tI, nI, pr=False):
    nC = nI*(tI-tH)/(tC-tH)
    nH = nI-nC
    flag = nR >= 0 and nH >= 0 and nC >= 0
    handle.x = [nR] if flag else []
    handle.y = [tI] if flag else []
    handle.z = [nI] if flag else []
    if pr:
        print(nR, tI, nI, nH, nC)

def update(pF=tF0, tF=tF0, pT=pT0, tT=tT0):
    with fig.batch_update():
        nF = moles(pF, vF, tF)
        nT = moles(pT, vF, tT)
        fig.update_layout(
            xaxis_range=[0, nF],
            yaxis_range=[tC, tH])
        # Surface
        nR = np.linspace(0, nF, SN)
        tI = np.linspace(tC, tH, SN)
        nR, tI = np.meshgrid(nR, tI)
        nI = (tT*nT-tF*(nF-nR))/tI
        m = mask(nF, nR, tI, nI)
        nR[np.where(m)] = np.nan
        S.x = nR[0,:]
        S.y = tI[:,0]
        S.z = nI
        # Embedding
        nR = np.linspace(0, nF, EN)
        nI = nT-nF+nR
        tI = (tT*nT-tF*(nF-nR))/nI
        m = mask(nF, nR, tI, nI)
        nR[np.where(m)] = np.nan
        E.x = nR
        E.y = tI
        E.z = nI
        #! NOTE:
        #! - These are the only values we need in the end
        #! - Three non-coexisting solutions, optimizing for no removal, and then cold gas
        # nR=0 point
        nR = 0
        tI = (tT*nT-tF*nF)/(nT-nF)
        nI = nT-nF
        point(A, nR, tI, nI)
        # tI=tH point
        tI = tH
        nR = nF-(nT*(tT-tH))/(tF-tH)
        nI = nT-nF+nR
        point(B, nR, tI, nI)
        # tI=tC point
        tI = tC
        nR = nF-(nT*(tT-tC))/(tF-tC) # this is completely fucked up somehow
        nI = nT-nF+nR
        point(C, nR, tI, nI, True)
        
layout = Layout(width='100%', margin='0 0 0 0')
interact(update,
        pF=FloatSlider(value=pF0, min=pMin, max=pMax, step=50, layout=layout),
        tF=FloatSlider(value=tF0, min=tC,   max=tH,   step=10, layout=layout),
        pT=FloatSlider(value=pT0, min=pMin, max=pMax, step=50, layout=layout),
        tT=FloatSlider(value=tT0, min=tC,   max=tH,   step=10, layout=layout))

fig

interactive(children=(FloatSlider(value=6000.0, description='pF', layout=Layout(margin='0 0 0 0', width='100%'…

FigureWidget({
    'data': [{'opacity': 0.5,
              'showscale': False,
              'type': 'surface'…